In [9]:
import simpy
import random

class MachinePark:

  def __init__(self, env):

    #CONSTANTS
    self.windows_made = 0
    self.hours = 8                  #working hours
    self.days = 23                  #business days
    self.total_time = self.hours * self.days  #total working time (hours)
    self.dispatch_capacity = 500    #dispatch

    #RESOURCES
    #wood
    self.wood_capacity = 500
    self.initial_wood = 200

    #glass
    self.glass_capacity = 100
    self.initial_glass = 60

    #paint
    self.front_wing_pre_paint_capacity = 60
    self.back_wing_pre_paint_capacity = 60
    self.front_wing_post_paint_capacity = 120
    self.back_wing_post_paint_capacity = 120

    #employees per activity
    #front_wing
    self.num_front_wing = 2 #number of assigned persons
    self.mean_front_wing = 1  #mean time for creating 1 piece perr 1 pers
    self.std_front_wing = 0.1  #standard dev

    #back_wing
    self.num_back_wing = 1
    self.mean_back_wing = 1
    self.std_back_wing = 0.2

    #paint
    self.num_paint = 3
    self.mean_paint = 3
    self.std_paint = 0.3

    #ensambling
    self.num_ensam = 2
    self.mean_ensam = 1
    self.std_ensam = 0.2

    #critical levels (critical stock should be 1 business day greater than supplier take to come)
    self.wood_critial_stock = (((8/self.mean_front_wing) * self.num_front_wing + (8/self.mean_back_wing) * self.num_back_wing) * 3) #2 days to deliver + 1 extra
    self.glass_critical_stock = (8/self.mean_ensam) * self.num_ensam * 2 #1 day to deliver + 1 extra

    #SimPy Containers
    self.wood = simpy.Container(env, capacity = self.wood_capacity, init = self.initial_wood)
    self.glass = simpy.Container(env, capacity = self.glass_capacity, init = self.initial_glass)
    self.front_wing_pre_paint = simpy.Container(env, capacity = self.front_wing_pre_paint_capacity, init = 0)
    self.back_wing_pre_paint = simpy.Container(env, capacity = self.back_wing_pre_paint_capacity, init = 0)
    self.front_wing_post_paint = simpy.Container(env, capacity = self.front_wing_post_paint_capacity, init = 0)
    self.back_wing_post_paint = simpy.Container(env, capacity = self.back_wing_post_paint_capacity, init = 0)
    self.dispatch = simpy.Container(env ,capacity = self.dispatch_capacity, init = 0)

    #SimPy Processes
    self.wood_control = env.process(self.wood_stock_control(env))
    self.glass_control = env.process(self.glass_stock_control(env))
    self.dispatch_control = env.process(self.dispatch_windows_control(env))


  #Bound Methods
  def wood_stock_control(self, env):
      yield env.timeout(0)
      while True:
          if self.wood.level <= self.wood_critial_stock:
              print('wood stock bellow critical level ({0}) at day {1}, hour {2}'.format(
                  self.wood.level, int(env.now/8), env.now % 8))
              print('calling wood supplier')
              print('----------------------------------')
              yield env.timeout(16)
              print('wood supplier arrives at day {0}, hour {1}'.format(
                  int(env.now/8), env.now % 8))
              yield self.wood.put(300)
              print('new wood stock is {0}'.format(
                  self.wood.level))
              print('----------------------------------')
              yield env.timeout(8)
          else:
              yield env.timeout(1)
  
  def glass_stock_control(self, env):
      yield env.timeout(0)
      while True:
          if self.glass.level <= self.glass_critical_stock:
              print('glass stock bellow critical level ({0}) at day {1}, hour {2}'.format(
                  self.glass.level, int(env.now/8), env.now % 8))
              print('calling glass supplier')
              print('----------------------------------')
              yield env.timeout(9)
              print('glass supplier arrives at day {0}, hour {1}'.format(
                  int(env.now/8), env.now % 8))
              yield self.glass.put(30)
              print('new glass stock is {0}'.format(
                  self.glass.level))
              print('----------------------------------')
              yield env.timeout(8)
          else:
              yield env.timeout(1)
    
  def dispatch_windows_control(self, env):
          global windows_made
          yield env.timeout(0)
          while True:
              if self.dispatch.level >= 50:
                  print('dispach stock is {0}, calling store to pick windows at day {1}, hour {2}'.format(
                      self.dispatch.level, int(env.now/8), env.now % 8))
                  print('----------------------------------')
                  yield env.timeout(4)
                  print('store picking {0} windows at day {1}, hour {2}'.format(
                      self.dispatch.level, int(env.now/8), env.now % 8))
                  self.windows_made += self.dispatch.level
                  yield self.dispatch.get(self.dispatch.level)
                  print('----------------------------------')
                  yield env.timeout(8)
              else:
                  yield env.timeout(1)


  #Main Processes #Generators (Use Main Processes and utilize Available Resources, adds stochastic variation)
  def front_wing_maker(self, env):
      while True:
          yield self.wood.get(1)
          self.front_wing_time = random.gauss(self.mean_front_wing, self.std_front_wing)
          yield env.timeout(self.front_wing_time)
          yield self.front_wing_pre_paint.put(1)

  def front_wing_maker_gen(self, env):
      for i in range(self.num_front_wing):
          env.process(self.front_wing_maker(env))
          yield env.timeout(0)

  def back_wing_maker(self, env):
    while True:
        yield self.wood.get(1)
        self.back_wing_time = random.gauss(self.mean_back_wing, self.std_back_wing)
        yield env.timeout(self.back_wing_time)
        yield self.back_wing_pre_paint.put(2)

  def back_wing_maker_gen(self, env):
      for i in range(self.num_back_wing):
          env.process(self.back_wing_maker(env))
          yield env.timeout(0)

  def painter(self, env):
      while True:
          yield self.front_wing_pre_paint.get(5)
          yield self.back_wing_pre_paint.get(5)
          self.paint_time = random.gauss(self.mean_paint, self.std_paint)
          yield env.timeout(self.paint_time)
          yield self.front_wing_post_paint.put(5)
          yield self.back_wing_post_paint.put(5)

  def painter_maker_gen(self, env):
      for i in range(self.num_paint):
          env.process(self.painter(env))
          yield env.timeout(0)

  def assembler(self, env):
      while True:
          yield self.front_wing_post_paint.get(1)
          yield self.back_wing_post_paint.get(1)
          yield self.glass.get(1)
          self.assembling_time = max(random.gauss(self.mean_ensam, self.std_ensam), 1)
          yield env.timeout(self.assembling_time)
          yield self.dispatch.put(1)

  def assembler_maker_gen(self, env):
      for i in range(self.num_ensam):
          env.process(self.assembler(env))
          yield env.timeout(0)


  def run_machine_park(self, env):
    self.front_wing_gen = env.process(self.front_wing_maker_gen(env))
    self.back_wing_gen = env.process(self.back_wing_maker_gen(env))
    self.painter_gen = env.process(self.painter_maker_gen(env))
    self.assembler_gen = env.process(self.assembler_maker_gen(env))

    env.run(until = self.total_time)

    print('Pre paint has {0} bodies and {1} back_wings ready to be painted'.format(
        self.front_wing_pre_paint.level, self.back_wing_pre_paint.level))
    print('Post paint has {0} bodies and {1} back_wings ready to be assembled'.format(
        self.front_wing_post_paint.level, self.back_wing_post_paint.level))
    print(f'Dispatch has %d windows ready to go!' % self.dispatch.level)
    print(f'----------------------------------')
    print('total windows made: {0}'.format(self.windows_made + self.dispatch.level))
    print(f'----------------------------------')
    print(f'SIMULATION COMPLETED')


In [10]:
#NEW RUN

#Create Environment
env = simpy.Environment(initial_time=0)

#Create factory object
machine_park = MachinePark(env)

machine_park.run_machine_park(env)

glass stock bellow critical level (31) at day 2, hour 5
calling glass supplier
----------------------------------
glass supplier arrives at day 3, hour 6
new glass stock is 44
----------------------------------
dispach stock is 51, calling store to pick windows at day 4, hour 2
----------------------------------
glass stock bellow critical level (29) at day 4, hour 6
calling glass supplier
----------------------------------
store picking 59 windows at day 4, hour 6
----------------------------------
wood stock bellow critical level (70) at day 5, hour 3
calling wood supplier
----------------------------------
glass supplier arrives at day 5, hour 7
new glass stock is 43
----------------------------------
glass stock bellow critical level (28) at day 6, hour 7
calling glass supplier
----------------------------------
wood supplier arrives at day 7, hour 3
new wood stock is 323
----------------------------------
glass supplier arrives at day 8, hour 0
new glass stock is 41
--------------

Bad pipe message: %s [b'\xc5\x96-\x86\xc5\xc1\x883\xc9\xde.i\xd0\xeb\x83\xf35C \x12\xeb\xb0\xa8\xad\x1aa\xc0\xd2\x7f\x90\x8e$r\x81%A\x0c[*\xfc\xc3\x17\xe7`j\xdf\xfb|\x11\xb84\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16']
Bad pipe message: %s [b"\x00\\\xf4\x05Qk\r\x10t\x96n\x9c\xfcQ)\x7fZN\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j"]
Bad pipe message: %s [b"~Mi\x0c\x10\x1f\x02\x95P\x95\x9d\x98\x85\xa0\x15tX\x86\x00\x00\xa6\xc0,\xc00\x00\xa3\x00\x9

In [7]:
# /** Break the machine every now and then. */
class MachineWear(val machine: Machine, val repairMan: Resource) : Component(process = MachineWear::breakMachine) {


    fun breakMachine(): Sequence<Component> = sequence {
        val timeToFailure = exponential(MTTF)

        while (true) {
            hold(timeToFailure())

            // handle the rare case that the model
            if (machine.isInterrupted) continue

            machine.interrupt()

            request(repairMan)
            hold(REPAIR_TIME)

            require(!isBumped(repairMan)) { "productive tools must not be bumped" }

            release(repairMan)

            machine.resume()
            require(!machine.isInterrupted) { "machine must not be interrupted at end of wear cycle" }
        }
    }
}

SyntaxError: expected ':' (4143005630.py, line 2)